### Script to plot latent heating PV tendency profiles ###

#### *Fig 11 in JAMES paper*
---

Manda Chasteen<br>
April 2024

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import glob

np.set_printoptions(suppress=True)

import warnings
warnings.simplefilter("ignore") 

In [ ]:
# Global mean cell height (AGL) -- precomputed: 

hgt_mean = np.array([23.46662 , 76.577835, 142.2688 , 220.98576 , 313.19623 , 419.373 , 539.91125 , 675.28613 , 825.9186 , 992.19366 , 
            1174.7301 , 1373.7909 , 1589.8113 , 1823.2872 , 2074.7295 , 2344.283 , 2632.389 , 2939.8752 , 3266.5732 , 
            3613.3872 , 3980.574 , 4368.4404 , 4777.5244 , 5208.391 , 5660.9995 , 6136.505 , 6634.999 , 7157.0967 , 
            7703.0933 , 8273.639 , 8869.326 , 9491.046 , 10139.161 , 10814.469 , 11517.821 , 12248.614 , 13009.587 , 
            13799.668 , 14619.748 , 15471.564 , 16354.052 , 17263.8 , 18186.29 , 19110.822 , 20036.516 , 20962.666 , 
            21889.504 , 22815.92 , 23743.23 , 24668.701 , 25595.904 , 26523.398 , 27450.395 , 28378.082 , 29305.627 ])

In [ ]:
variables = ['depv_dt_cu', 'depv_dt_mp', 'depv_dt_mp_evap_cw', 'depv_dt_mp_evap_rw', 'depv_dt_mp_depo_ice',
             'depv_dt_mp_frez_ice', 'depv_dt_mp_melt_ice']

parent = ''      # file directory 

files = glob.glob(parent+'pvbudget15to3km_double_fixed_20190520_03:00:36*')
file = xr.open_mfdataset(files, parallel=True, chunks={'longitude' : 100, 'latitude' : 100, 'nVertLevels' : 5})[variables]

In [ ]:
### Point 1: within refined mesh region

lat1 = 37.6 
lon1 = -86.2

f_pt1 = file.sel(longitude=[lon1], latitude=[lat1], method="nearest").squeeze()

### Point 2: within coarse mesh region

lat2 = 40.25
lon2 = 124.0

f_pt2 = file.sel(longitude=[lon2], latitude=[lat2], method="nearest").squeeze()

In [ ]:
# Make 2-panel plot of PV tendencies from various microphysics processes + cumulus scheme:

levels = np.arange(1,56,1)
hgt_km = hgt_mean / 1000


plt.rcParams['savefig.dpi'] = 350
fig, axs = plt.subplots(1,2,figsize=(15,7)) 
fig.patch.set_facecolor('white')
fig.patch.set_alpha(1)
ax = axs.flatten()

# Set up common plot attributes:
for i in np.arange(0,np.shape(ax)[0],1):

    # Axis limits and y ticks:
    ax[i].set_ylim([0,40])
    ytix = np.hstack((1, np.arange(5,45,5)))
    ax[i].set_yticks(ytix)

    # Rasterization order:
    ax[i].set_rasterization_zorder(1)

    # Add horizontal lines:
    ax[i].hlines(1, -0.08, 0.08, zorder=1.2, linewidth=0.25, color='gray')
    ax[i].vlines(0, 0, 57, zorder=1.3, linewidth=0.5, color='black')

    # Add y axis for mean height AGL
    axa = ax[i].twinx()
    axa.set_ylim(ax[i].get_ylim())
    axa.set_yticks(np.arange(5,40,5))
    axa.set_yticks(np.hstack((1,np.arange(5,45,5))))
    labels = [np.round(hgt_km[lev-1],1) for lev in axa.get_yticks()]
    axa.set_yticklabels(labels)

    if (i % 2) == 0:
        ax[i].set_ylabel('Model Level', fontsize=11.5, labelpad=6.5)
    else:
        axa.set_ylabel('Mean Height AGL [km]', fontsize=11.5, labelpad=6.5)


# Point 1:
ax[0].plot(f_pt1.depv_dt_cu, levels, c='grey', linewidth=2, alpha=1, zorder=2)
ax[0].plot(f_pt1.depv_dt_mp_evap_rw, levels, c='#0D9276', linewidth=2, zorder=2)
ax[0].plot(f_pt1.depv_dt_mp_evap_cw, levels, c='#0766AD', linewidth=2, zorder=2)
ax[0].plot(f_pt1.depv_dt_mp_depo_ice, levels, c='#E96479', linewidth=2, zorder=2)
ax[0].plot(f_pt1.depv_dt_mp_melt_ice, levels, c='#AF2655', linewidth=2, zorder=2)
ax[0].plot(f_pt1.depv_dt_mp_frez_ice, levels, c='#492E87', linewidth=2, zorder=2)
ax[0].plot(f_pt1.depv_dt_mp, levels, c='black', linewidth=2, alpha=0.7, zorder=2)
ax[0].plot(f_pt1.depv_dt_mp+f_pt1.depv_dt_cu, levels, c='red', linewidth=2, alpha=0.7, zorder=2)

ax[0].set_xlim([-0.03, 0.06])
ax[0].set_title(r'PV Tendency from Latent Heating Processes',fontsize=12,y=1.02,fontweight='medium', loc='left', horizontalalignment='left')
ax[0].set_title(str(lat1).zfill(2)+'$^{\circ}$ N, '+str(lon1).zfill(2)+'$^{\circ}$ W',fontsize=10,y=1.02,fontweight='medium', c='dimgray', loc='right', horizontalalignment='right')


# Point 2:
ax[1].plot(f_pt2.depv_dt_cu, levels, c='grey', linewidth=2, alpha=1, zorder=2, label='Cumulus')
ax[1].plot(f_pt2.depv_dt_mp_evap_rw, levels, c='#0D9276', linewidth=2, zorder=2, label='Rain Evaporation')
ax[1].plot(f_pt2.depv_dt_mp_evap_cw, levels, c='#0766AD', linewidth=2, zorder=2, label='Condensation/Evaporation')
ax[1].plot(f_pt2.depv_dt_mp_depo_ice, levels, c='#E96479', linewidth=2, zorder=2, label='Deposition/Sublimation')
ax[1].plot(f_pt2.depv_dt_mp_melt_ice, levels, c='#AF2655', linewidth=2, zorder=2, label='Melting')
ax[1].plot(f_pt2.depv_dt_mp_frez_ice, levels, c='#492E87', linewidth=2, zorder=2, label='Freezing')
ax[1].plot(f_pt2.depv_dt_mp, levels, c='black', linewidth=2, alpha=0.7, zorder=2, label='All Microphysics')
ax[1].plot(f_pt2.depv_dt_mp+f_pt2.depv_dt_cu, levels, c='red', linewidth=2, alpha=0.7, zorder=2, label='All Latent Heating')


ax[1].set_xlim([-0.04, 0.03])
ax[1].set_title(r'PV Tendency from Latent Heating Processes',fontsize=12,y=1.02,fontweight='medium', loc='left', horizontalalignment='left')
ax[1].set_title(str(lat2).zfill(2)+'$^{\circ}$ N, '+str(lon2).zfill(2)+'$^{\circ}$ E',fontsize=10,y=1.02,fontweight='medium', c='dimgray', loc='right', horizontalalignment='right')
ax[1].legend()

fig.subplots_adjust(bottom=0.1, top=0.9, left=0.1, right=0.9, wspace=0.3, hspace=0.4)


figname = 'mpas_mp_point_profiles_regrid_03.00.18Z.pdf'
#fig.savefig(figname,bbox_inches="tight")

plt.show()
